In [ ]:
import nltk
nltk.download('stopwords')
!pip install --user -U nltk
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')

import pandas as pd
import numpy as np
import re
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
import matplotlib
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
import nltk
import nltk.data
import numba
import seaborn as sns
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer

This is establishing the preprocessing part

In [2]:

def preprocessing(text):
    # text=text[0:100]
    #Lowercasing the text
    lower=text.lower()

    #Splitting into tokens
    tokens=lower.split()

    #Removing stop words
    without_stop = []
    for tok in tokens:
        if tok not in en_stopwords:
            without_stop.append(tok)

    #Removing Punctuations and Blank Space tokens
    tokenizer = RegexpTokenizer(r"\w+")
    final_words=tokenizer.tokenize(' '.join(without_stop))
    
    return final_words

Starting the code for Boolean Queries

In [14]:
import pickle
import copy 

uni_invert_index = open('D:/College/SEMESTER 6/IR/CSE508_Winter2023_A1_104/Unigram_Inverted_Index_File', 'rb')     
word_dict = pickle.load(uni_invert_index)
uni_invert_index.close()

def create_querry(words, operations):
  ans = ""
  for i in range(len(words)-1):
    ans += words[i]+" "+operations[i]+" "
  ans = ans+words[-1]
  return ans

def name_and(word3,file_list):
    filenames = []
    filelist =[]
    list_for_doc1 = word_dict[word3][1]
    num_comparison = min(len(list_for_doc1), len(file_list))
    for key in list_for_doc1:
        if key in file_list:
            filelist.append(key)
            name = "cranfield"+ "0"*(4-len(str(key))) + str(key)
            filenames.append( name )
    return filenames, filelist, num_comparison

def name_or(word3,file_list):
    filelist = copy.deepcopy(file_list)
    list_for_doc1 = word_dict[word3][1]
    filenames = []
    num_comparison = len(filelist) + len(list_for_doc1)
    for key in list_for_doc1:
        if(key not in filelist):
          filelist.append(key)     
    for key in filelist:
      name = "cranfield"+ "0"*(4-len(str(key))) + str(key)
      filenames.append(name)
    return filenames, filelist, num_comparison

In [31]:
def name_and_not(word3, file_list):
    all_files_list = []
    for s in range(1400):
        all_files_list.append(s+1)
    _file_list = copy.deepcopy(file_list)
    filelist = []
    filenames = []
    list_for_doc1 = word_dict[word3][1]
    
    num_comp = min(len(_file_list), min(list_for_doc1), 1400-len(_file_list), 1400-len(list_for_doc1))
    
    for key in _file_list:
        if key in list_for_doc1:
            _file_list.remove(key)
            
    for key in _file_list:
        name = "cranfield"+ "0"*(4-len(str(key))) + str(key)
        filenames.append(name)
    return filenames, _file_list, num_comp

def name_or_not(word3, file_list):
    all_files_list = []
    for s in range(1400):
        all_files_list.append(s+1)
    filelist = copy.deepcopy(file_list)
    list_for_doc1 = word_dict[word3][1]
    not_list_for_doc1 = all_files_list
    filenames = []
    for key in list_for_doc1:
            not_list_for_doc1.remove(key)
    num_comparison = len(filelist) + len(not_list_for_doc1)
    for key in not_list_for_doc1:
        if key not in filelist:
            filelist.append(key)
    for key in filelist:
        name = "cranfield"+ "0"*(4-len(str(key))) + str(key)
        filenames.append(name)
    return filenames, filelist, num_comparison

In [16]:
# def binary_search(arr, x):
#     comparisons = 0
#     left = 0
#     right = len(arr) - 1
#     while left <= right:
#         mid = (left + right) // 2
#         comparisons += 1
#         if arr[mid] == x:
#             return comparisons
#         elif arr[mid] < x:
#             left = mid + 1
#         else:
#             right = mid - 1
#     return -1


# def total_comparisons(docs, query):
#     total_comps = 0
#     for word in query.split():
#         if word in docs:
#             total_comps += binary_search(docs[word], query)
#     return total_comps 

In [3]:
def take_input():
    seq = input("Enter input seq: ")
    operations = [x for x in input("Enter commma seperated operation: ").split(',')]
    lis = preprocessing(seq)
    # querry = create_querry(lis,operations)
    return lis, operations

Forming the structure of the queries

In [23]:
print("INPUT:")
num_queries = int(input("Enter the number of queries you want to perform: "))
print("Number of queries is: " + str(num_queries))
list_of_input_words = []
list_of_input_operators = []
for j in range(num_queries):
    input_words, input_operators = take_input()
    print(str(j+1) + ". Input Sequence is:", end= " ")
    print(input_words)
    print(str(j+1) + ". List of operators is:", end=" ")
    print(input_operators)
    list_of_input_words.append(input_words)
    list_of_input_operators.append(input_operators)

INPUT:
Number of queries is: 1
1. Input Sequence is: ['drops', 'fifth', 'inspection', 'similitude']
1. List of operators is: ['AND', 'OR', 'AND NOT']


In [33]:
for k in range(num_queries):
    words_k = list_of_input_words[k]
    operators_k = list_of_input_operators[k]
    query_k = create_querry(words_k, operators_k)
    print("Query " + str(k+1) + ":", end = " ")
    print(query_k)
    num_op = len(operators_k)
    list1 = word_dict[words_k[0]][1]
    name_docs =[]
    number_of_comparisons = 0
    
    for op in range(num_op):

        if(operators_k[op] == "AND"):
          cur_name_docs,list1, num_com = name_and(words_k[op+1], list1)
          name_docs = cur_name_docs
          number_of_comparisons = number_of_comparisons + num_com
          
        elif(operators_k[op] == "OR"):
          cur_name_docs, list1, num_com = name_or(words_k[op+1], list1)
          name_docs = cur_name_docs
          number_of_comparisons = number_of_comparisons + num_com
          
        elif(operators_k[op] == "AND NOT"):
          cur_name_docs, list1, num_com = name_and_not(words_k[op+1], list1)
          name_docs = cur_name_docs
          number_of_comparisons = number_of_comparisons + num_com
               
        elif(operators_k[op] == "OR NOT"):
          cur_name_docs, list1, num_com = name_or_not(words_k[op+1], list1)
          name_docs = cur_name_docs
          number_of_comparisons = number_of_comparisons + num_com
      
    name_docs_set = [*set(name_docs)]      
    print("Number of documents retrieved for query " + str(k+1) + ":", end=" ")
    print(len(name_docs_set))
    print("Name of the documents retrieved for query " + str(k+1) + ":", end = " ")
    for items in name_docs_set:
      print(items + ",", end = " ")
    print(" ")
    print("Number of comparisons required for query " + str(k+1) + ": " + str(number_of_comparisons))    

Query 1: drops AND fifth OR inspection AND NOT similitude
Number of documents retrieved for query 1: 2
Name of the documents retrieved for query 1: cranfield1033, cranfield0730,  
Number of comparisons required for query 1: 10
